In [1]:
# chargement des données
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from scipy.stats import chi2_contingency
import scipy.stats as stats
import os
import math

import warnings
warnings.filterwarnings("ignore")

In [2]:
hackhaton=pd.read_csv("hackhaton_logic_replaced_fin.csv",encoding='ISO-8859-1')
hackhaton.drop('Unnamed: 0', axis =1, inplace=True)
hackhaton

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,adjpat2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,adjpat2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,adjpat2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,adjpat2c


# Normalisation des libellés (pas de caractères spéciaux, ni d'espaces, ni de voyelles)

In [3]:
hackhaton2=hackhaton


In [4]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,adjointadministratif.........
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,adjointadministratif.........
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,adjpat2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,adjpat2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,adjpat2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,adjpat2c


In [5]:
hackhaton2['LIBELLE_EMPLOI_GRADE']=[str(i) for i in hackhaton2['LIBELLE_EMPLOI_GRADE']]

In [6]:
#on enleve les voyelles des libellés ? (si jamais il y a des abreviations, souvent on saute les voyelles...) 
#On enleve aussi les espaces on sait jamais

voyelles='aeiouyÃÂ'

L=[]
for i in hackhaton2['LIBELLE_EMPLOI_GRADE'] :
    
    if i != 'nan' :
        m=''
        
        for j in range(0, len(i)):
            
            if i[j] in voyelles:  #Enlever les voyelles (pour les abréviations) 
                pass
            
            else :
                m = m + i[j]
        
        
    else :
        m=i
    
    m = "".join(char for char in m if char.isalnum()) #Enlever les caractères spéciaux
    L.append(m)

In [7]:
hackhaton2['LIBELLE_EMPLOI_GRADE']=[str(i) for i in L]

In [8]:
hackhaton2

,ID_CARRIERE,date_debut,date_fin,ETAT,CATEGORIE_EMPLOI,QUALITE,STATUT_AGENT,INDICE_BRUT,EMPLOI_STATUTAIRE,LIBELLE_EMPLOI_GRADE
0,56,01/01/1999,26/03/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
1,56,27/03/1999,16/07/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
2,56,17/07/1999,31/12/1999,1,A,T,T,NaN,NaN,djntdmnstrtf
3,56,01/01/2000,02/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
4,56,03/07/2000,19/07/2000,1,A,T,T,NaN,NaN,djntdmnstrtf
...,...,...,...,...,...,...,...,...,...,...
1048570,1861885,01/02/2009,28/02/2009,1,A,T,T,303.0,NaN,djpt2c
1048571,1861885,01/03/2009,31/03/2009,1,A,T,T,303.0,NaN,djpt2c
1048572,1861885,01/04/2009,08/04/2009,1,A,T,T,303.0,NaN,djpt2c
1048573,1861885,09/04/2009,19/04/2009,1,A,T,T,303.0,NaN,djpt2c


# Fonction de ressemblance des libellés

## Séparation des libellés associés a des nan et les autres 

In [9]:
# liste des libellés associés a des 'nan'
ind_code_nan=[i for i in range (0,1048574) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) == 'nan']
libelle_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_nan]
libelle_nan=list(set(libelle_nan))
len(libelle_nan)

7831

In [10]:
# liste des libellés  (unique) associés a des codes (dictionnaire ?)

ind_code_no_nan=[i for i in range (0,1048574) if str(hackhaton2.EMPLOI_STATUTAIRE[i]) != 'nan']
libelle_no_nan =[hackhaton2.LIBELLE_EMPLOI_GRADE[i] for i in ind_code_no_nan]
libelle_no_nan=list(set(libelle_no_nan))
len(libelle_no_nan)

7501

## Fonction de ressemblance

In [11]:
import difflib


#L=[corresondant de libelle_nan[i], si pas de correspondant, c'est libelle_nan[i]]
# de longueur len(libelle_nan) 
c1=0
c2=0

L=[]
for i in range (0,len(libelle_nan)) : 
    print(i,'/',len(libelle_nan))
    Match=difflib.get_close_matches(libelle_nan[i], libelle_no_nan, cutoff = 0.95) #On selectionne les libellé assigné a un code qui match a au moins 90% avec notre libellé sans code 

    if len(Match) != 0:
        c1=c1+1
        Match_ratio = [difflib.SequenceMatcher(None, libelle_nan[i], a).ratio() for a in Match]
        index_max= Match_ratio.index(max(Match_ratio))

        L.append(Match[index_max]) #Match[index_max] est l'element le plus ressemblant a notre libelle de base
        
    else : 
        c2=c2+1
        L.append(libelle_nan[i])

0 / 7831
1 / 7831
2 / 7831
3 / 7831
4 / 7831
5 / 7831
6 / 7831
7 / 7831
8 / 7831
9 / 7831
10 / 7831
11 / 7831
12 / 7831
13 / 7831
14 / 7831
15 / 7831
16 / 7831
17 / 7831
18 / 7831
19 / 7831
20 / 7831
21 / 7831
22 / 7831
23 / 7831
24 / 7831
25 / 7831
26 / 7831
27 / 7831
28 / 7831
29 / 7831
30 / 7831
31 / 7831
32 / 7831
33 / 7831
34 / 7831
35 / 7831
36 / 7831
37 / 7831
38 / 7831
39 / 7831
40 / 7831
41 / 7831
42 / 7831
43 / 7831
44 / 7831
45 / 7831
46 / 7831
47 / 7831
48 / 7831
49 / 7831
50 / 7831
51 / 7831
52 / 7831
53 / 7831
54 / 7831
55 / 7831
56 / 7831
57 / 7831
58 / 7831
59 / 7831
60 / 7831
61 / 7831
62 / 7831
63 / 7831
64 / 7831
65 / 7831
66 / 7831
67 / 7831
68 / 7831
69 / 7831
70 / 7831
71 / 7831
72 / 7831
73 / 7831
74 / 7831
75 / 7831
76 / 7831
77 / 7831
78 / 7831
79 / 7831
80 / 7831
81 / 7831
82 / 7831
83 / 7831
84 / 7831
85 / 7831
86 / 7831
87 / 7831
88 / 7831
89 / 7831
90 / 7831
91 / 7831
92 / 7831
93 / 7831
94 / 7831
95 / 7831
96 / 7831
97 / 7831
98 / 7831
99 / 7831
100 / 7831

In [12]:
print(c1 , 'de remplacés et ', c2, 'non remplacés sur en tout', c1+c2, 'libellés') 

3944 de remplacés et  3887 non remplacés sur en tout 7831 libellés


In [13]:
len(libelle_nan) == len(L) 
#Les deux listes pour creer le dictionnaire de correspondance entre libellé avec des nan et leur libellé (qui ressemble a 90% et avec un code) correspondant

True

In [14]:
correspondance=[{libelle_nan[i] : L[i]} for i in range (0,len(L))]

In [15]:
c=correspondance[0]
for i in range (1,len(correspondance)): 
    c.update(correspondance[i])


In [16]:
c #Dictionnaire a appliquer a la colonne LIBELLE 

{'': '',
 'dsgnntstgr': 'dsgnntstgr',
 'sprpmpr2mclss': 'sprpmprd2mclss',
 'p2': 'p2',
 'mntrddphs1ct': 'mntrddphs1ct',
 'cndctrsp2nv': 'cndctrsp2nv',
 'djdmpl2cl': 'djdmpl2cl',
 'rspnsblmntnncbtm': 'rspnsblmntnncbtm',
 'tchnlbrtrcn': 'tchlbrtrcn',
 'mnplctrsrvllnt': 'mnplctrsrvllnt',
 'ssstntscdctftrr': 'ssstntscdctftrr',
 'gntdbrdctlgrph': 'gntdbrdctlgrph',
 'dctrps2mclss': 'dctrps2mclss',
 'vrrprf1ctgr': 'vrrprf1ctgr',
 'vrrpqmcn': 'vrrpqmcn',
 'ddsn2': 'ddsn2',
 'djnttchnprncp2mcl': 'djnttchnprncp2mcl',
 'trrssr': 'trrssr',
 'msrknsthrpt': 'mssrknsthrpt',
 'nfrmrsnsgnrxcnrl': 'nfrmrsnsgnrxcnrl',
 'ngnrnchfclssnrml': 'ngnrnchfdclssnrml',
 'scrtrdmclspdmpr': 'scrtrdmclspdmpr',
 'dsg': 'dsg',
 'sfmm': 'sfmm',
 'lvnfrmrdpschtr': 'lvnfrmrdpschtr',
 'djnttchnqp1c': 'djnttchnq1c',
 'prprtr': 'prprtr',
 'djnttchnqtrrtrld2cls': 'djnttchnqtrrtrld2cls',
 'grdndms': 'grdndms',
 'djdmnhsp1rcl': 'djdmnhsp1rcl',
 'rspnsblctm': 'rspnsblctm',
 'gtsrvhsp': 'gtsrvhsp',
 'gntsrvntrr2mct': 'gntsrvntrr2

## Application a la BDD

# code pour : égalité des libellé implique changement de code 